In [1]:
#| default_exp utils/plot

In [ ]:
#| export
import matplotlib.pyplot as plt

In [ ]:
#| export
def savefig(name, **kwargs):
    plt.savefig(f"../figures/{name}.png", bbox_inches="tight", **kwargs)
    plt.savefig(f"../figures/{name}.pdf", bbox_inches="tight", **kwargs)

### MVA plotting

In [2]:
#| export
import xarray as xr
import pandas as pd
from datetime import timedelta, datetime

from pyspedas.cotrans.minvar_matrix_make import minvar_matrix_make
from pyspedas import tvector_rotate
from pyspedas.analysis.tvectot import tvectot

from pytplot import tplot
from pytplot import store_data, get_data, split_vec, join_vec
from pytplot import timebar, highlight, degap, options


In [ ]:
#| export
import matplotlib.pyplot as plt
import scienceplots
from ids_finder.utils.plot import savefig

plt.style.use(['science', 'nature', 'notebook'])

In [ ]:
# | export
def time_stamp(ts):
    "Return POSIX timestamp as float."
    return pd.Timestamp(ts, tz="UTC").timestamp()


def setup_mva_plot(
    data: xr.DataArray,
    tstart: datetime,
    tstop: datetime,
    mva_tstart: datetime = None,
    mva_tstop: datetime = None,
):
    if mva_tstart is None:
        mva_tstart = tstart
    if mva_tstop is None:
        mva_tstop = tstop

    mva_b = data.sel(time=slice(mva_tstart, mva_tstop))
    store_data("fgm", data={"x": mva_b.time, "y": mva_b})
    minvar_matrix_make("fgm")  # get the MVA matrix

    temp_b = data.sel(time=slice(tstart, tstop))
    store_data("fgm", data={"x": temp_b.time, "y": temp_b})
    tvar = tvector_rotate("fgm_mva_mat", "fgm")[0]
    ysubtitle = "[nT LMN]"
    legend_names = [r"$B_l$", r"$B_m$", r"$B_n$"]
    
    tvar2plot = tvectot(tvar, join_component=True)
    legend_names = legend_names + [r"$B_{total}$"]

    options(tvar2plot, "ytitle", "$B$")
    options(tvar2plot, "ysubtitle", ysubtitle)
    options(tvar2plot, "legend_names", legend_names)
    
    options(tvar2plot, "thick", 2)
    options(tvar2plot, "char_size", 16)
    
    # tstart_ts = time_stamp(tstart)
    # tstop_ts = time_stamp(tstop)
    # highlight(tvar2plot, [tstart_ts, tstop_ts])
    degap(tvar2plot)
    return tvar2plot

In [ ]:
def format_candidate_title(candidate: dict):
    format_float = (
        lambda x: rf"$\bf {x:.2f} $" if isinstance(x, (float, int)) else rf"$\bf {x} $"
    )

    base_line = rf'$\bf {candidate.get("type", "N/A")} $ candidate (time: {candidate.get("time", "N/A")}) with index '
    index_line = rf'i1: {format_float(candidate.get("index_std", "N/A"))}, i2: {format_float(candidate.get("index_fluctuation", "N/A"))}, i3: {format_float(candidate.get("index_diff", "N/A"))}'
    info_line = rf'$B_n/B$: {format_float(candidate.get("BnOverB", "N/A"))}, $dB/B$: {format_float(candidate.get("dBOverB", "N/A"))}, $(dB/B)_{{max}}$: {format_float(candidate.get("dBOverB_max", "N/A"))},  $Q_{{mva}}$: {format_float(candidate.get("Q_mva", "N/A"))}'
    title = rf"""{base_line}
    {index_line}
    {info_line}"""
    return title

In [ ]:
#| export
def plot_candidate(candidate: dict, sat_fgm: xr.DataArray, **kwargs):
    if pd.notnull(candidate.get("d_tstart")) and pd.notnull(candidate.get("d_tstop")):
        tvar = setup_mva_plot(
            sat_fgm,
            candidate["tstart"],
            candidate["tstop"],
            candidate["d_tstart"],
            candidate["d_tstop"],
        )
    else:
        tvar = setup_mva_plot(sat_fgm, candidate["tstart"], candidate["tstop"])


    if "d_time" in candidate.keys():
        d_time_ts = time_stamp(candidate["d_time"])
        timebar(d_time_ts, color="red")
    if "d_tstart" in candidate.keys() and not pd.isnull(candidate["d_tstart"]):
        d_start_ts = time_stamp(candidate["d_tstart"])
        timebar(d_start_ts)
    if "d_tstop" in candidate.keys() and not pd.isnull(candidate["d_tstop"]):
        d_stop_ts = time_stamp(candidate["d_tstop"])
        timebar(d_stop_ts)

    return tplot(tvar, **kwargs)

In [ ]:

def plot_candidates(
    candidates: pd.DataFrame, candidate_type=None, num=4, plot_func=plot_candidate
):
    """Plot a set of candidates.

    Parameters:
    - candidates (pd.DataFrame): DataFrame containing the candidates.
    - candidate_type (str, optional): Filter candidates based on a specific type.
    - num (int): Number of candidates to plot, selected randomly.
    - plot_func (callable): Function used to plot an individual candidate.

    """

    # Filter by candidate_type if provided
    
    candidates = get_candidates(candidates, candidate_type, num)

    # Plot each candidate using the provided plotting function
    for _, candidate in candidates.iterrows():
        plot_func(candidate)

In [ ]:
def plot_basic(
    data: xr.DataArray, 
    tstart: datetime, 
    tstop: datetime,
    tau: timedelta, 
    mva_tstart=None, mva_tstop=None, neighbor: int = 1
):
    if mva_tstart is None:
        mva_tstart = tstart
    if mva_tstop is None:
        mva_tstop = tstop

    mva_b = data.sel(time=slice(mva_tstart, mva_tstop))
    store_data("fgm", data={"x": mva_b.time, "y": mva_b})
    minvar_matrix_make("fgm")  # get the MVA matrix

    temp_tstart = tstart - neighbor * tau
    temp_tstop = tstop + neighbor * tau

    temp_b = data.sel(time=slice(temp_tstart, temp_tstop))
    store_data("fgm", data={"x": temp_b.time, "y": temp_b})
    temp_btotal = calc_vec_mag(temp_b)
    store_data("fgm_btotal", data={"x": temp_btotal.time, "y": temp_btotal})

    tvector_rotate("fgm_mva_mat", "fgm")
    split_vec("fgm_rot")
    pytplot.data_quants["fgm_btotal"]["time"] = pytplot.data_quants["fgm_rot"][
        "time"
    ]  # NOTE: whenever using `get_data`, we may lose precision in the time values. This is a workaround.
    join_vec(
        [
            "fgm_rot_x",
            "fgm_rot_y",
            "fgm_rot_z",
            "fgm_btotal",
        ],
        new_tvar="fgm_all",
    )

    options("fgm", "legend_names", [r"$B_x$", r"$B_y$", r"$B_z$"])
    options("fgm_all", "legend_names", [r"$B_l$", r"$B_m$", r"$B_n$", r"$B_{total}$"])
    options("fgm_all", "ysubtitle", "[nT LMN]")
    tstart_ts = time_stamp(tstart)
    tstop_ts = time_stamp(tstop)
    # .replace(tzinfo=ZoneInfo('UTC')).timestamp()
    highlight(["fgm", "fgm_all"], [tstart_ts, tstop_ts])
    
    degap("fgm")
    degap("fgm_all")
